# Baseball Game Analysis

In this project, we will perform exploratory data analysis (EDA) and design a normalized database schema using a file of Major League Baseball games from Retrosheet. This project uses the following skills:
 - importing CSV data into a database
 - designing a normalized schema for a large, predominantly single table dataset
 - creating tables that match a schema design
 - migrating data from unnormalized tables into normalized tables

There are four main files used here:
 - `game_log.csv` was produced by combining 12 separate csv files from Retrosheet
 - `park_codes.csv`
 - `person_codes.csv`
 - `team_codes.csv`

In addition, the file `game_log_fields.txt` explains the fields included in the main file.

Because the main game log file is so large, we'll begin by setting pandas so that the DataFrame isn't truncated.

In [1]:
import pandas as pd

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [3]:
# Read the four csv files into DataFrames.
games = pd.read_csv('game_log.csv')
parks = pd.read_csv('park_codes.csv')
people = pd.read_csv('person_codes.csv')
teams = pd.read_csv('team_codes.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Now, let's explore the data.
games.shape

(171907, 161)

In [5]:
games.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [6]:
games.tail()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,4,60.0,D,NaN,NaN,NaN,DEN02,27762.0,203.0,0200000202,1100100010,39.0,10.0,4.0,1.0,2.0,6.0,0.0,0.0,1.0,4.0,0.0,12.0,2.0,1.0,0.0,0.0,8.0,7.0,4.0,4.0,1.0,0.0,30.0,12.0,1.0,0.0,0.0,0.0,41.0,13.0,4.0,0.0,1.0,4.0,1.0,0.0,1.0,3.0,0.0,11.0,0.0,1.0,0.0,0.0,12.0,5.0,6.0,6.0,0.0,0.0,30.0,13.0,0.0,0.0,0.0,0.0,barrs901,Scott Barry,woodt901,Tom Woodring,randt901,Tony Randazzo,ortir901,Roberto Ortiz,NaN,NaN,NaN,NaN,counc001,Craig Counsell,weisw001,Walt Weiss,thort001,Tyler Thornburg,rusic001,Chris Rusin,knebc001,Corey Knebel,susaa001,Andrew Susac,cravt001,Tyler Cravy,marqg001,German Marquez,villj001,Jonathan Villar,5.0,genns001,Scooter Gennett,4.0,cartc002,Chris Carter,3.0,santd002,Domingo Santana,9.0,pereh001,Hernan Perez,8.0,arcio002,Orlando Arcia,6.0,susaa001,Andrew Susac,2.0,elmoj001,Jake Elmore,7.0,cravt001,Tyler Cravy,1.0,blacc001,Charlie Blackmon,8.0,dahld001,David Dahl,7.0,arenn001,Nolan Arenado,5.0,gonzc001,Carlos Gonzalez,9.0,murpt002,Tom Murphy,2.0,pattj005,Jordan Patterson,3.0,valap001,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,5,51.0,D,NaN,NaN,NaN,PHI13,36935.0,159.0,000001100,00100031x,33.0,8.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,9.0,1.0,1.0,1.0,0.0,6.0,6.0,3.0,3.0,0.0,0.0,24.0,12.0,3.0,1.0,2.0,0.0,33.0,10.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,2.0,0.0,7.0,5.0,2.0,2.0,0.0,0.0,27.0,7.0,0.0,0.0,1.0,0.0,barkl901,Lance Barksdale,herna901,Angel Hernandez,barrt901,Ted Barrett,littw901,Will Little,NaN,NaN,NaN,NaN,collt801,Terr

In [7]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


The main game log file contains 161 columns and 171,907 rows, making this a very large file. It covers baseball games from 1871 to 2016. 

Columns 106 to 159 of the games file provide information about the starting players for both the visiting and home teams, listed in order (1-9) that they appeared in the batting order. The defensive position column contains numeric data that represents the following:
 - 1: pitcher
 - 2: catcher
 - 3: first baseman
 - 4: second baseman
 - 5: third baseman
 - 6: shortstop
 - 7: left fielder
 - 8: center fielder
 - 9: right fielder

In [8]:
parks.shape

(252, 9)

In [9]:
parks.head()

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


The parks file contains information on 252 baseball stadiums, including the stadium name, location, dates of use, and baseball league. The games file includes a `park_id` column, which could be used to join these two files together.

In [10]:
people.shape

(20494, 7)

In [11]:
people.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


The people file contains information on over 20,000 people in baseball, including players, managers, coaches, and umpires. The file includes first and last names and debut dates. The `id` column in the people file could be joined to the games file on the following columns:
 - `hp_umpire_id`
 - `1b_umpire_id`
 - `2b_umpire_id`
 - `3b_umpire_id`
 - `lf_umpire_id`
 - `rf_umpire_id`
 - `v_manager_id`
 - `h_manager_id`
 - `winning_pitcher_id`
 - `losing_pitcher_id`
 - `saving_pitcher_id`
 - `winning_rbi_batter_id`
 - `v_starting_pitcher_id`
 - `h_starting_pitcher_id`
 - `v_player_1_id` ... `v_player_9_id`
 - `h_player_1_id` ... `h_player_9_id`

In [12]:
teams.shape

(150, 8)

In [13]:
teams.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


The teams file contains information on 150 teams, including the baseball league, start and end date, city, team nickname (mascot), and franchise id. The `team_id` column in teams can be joined to the `v_name` and `h_name` columns in the main game log file. 

In [14]:
# Explore the leagues to see if the files are consistent
teams['league'].unique()

array(['UA', 'NL', 'PL', nan, 'AA', 'AL', 'FL'], dtype=object)

In [15]:
parks['league'].unique()

array(['NL', 'UA', 'AL', nan, 'AA', 'FL', 'PL'], dtype=object)

In [16]:
games['v_league'].unique()

array([nan, 'NL', 'AA', 'UA', 'PL', 'AL', 'FL'], dtype=object)

In [17]:
games['h_league'].unique()

array([nan, 'NL', 'AA', 'UA', 'PL', 'AL', 'FL'], dtype=object)

In [18]:
teams[teams['league'] == 'FL']

,team_id,league,start,end,city,nickname,franch_id,seq
10,BLF,FL,1914,1915,Baltimore,Terrapins,BLF,1
19,BRF,FL,1914,1915,Brooklyn,Tip-Tops,BRF,1
28,BUF,FL,1914,1915,Buffalo,Feds,BUF,1
33,CHF,FL,1914,1915,Chicago,Whales,CHF,1
61,IND,FL,1914,1914,Indianapolis,Hoosier-Feds,IND,1
62,NEW,FL,1915,1915,Newark,Peppers,IND,2
65,KCF,FL,1914,1915,Kansas City,Packers,KCF,1
104,PTF,FL,1914,1915,Pittsburgh,Rebels,PTF,1
120,SLF,FL,1914,1915,St. Louis,Terriers,SLF,1


All of the files contain the same leagues:
 - NL: National League (1876 - present)
 - AL: American League (1901 - present)
 - AA: American Association (1881–1891)
 - UA: Union Association (1884)
 - PL: Players' League (1890)
 - FL: Federal League (1914-1915)

In the United States and Canada, Major League Baseball (MLB) is currently divided into two leagues: the National League and the American League.

The other leagues listed are historical and not currently operational. The National League was founded in 1876 and replaced the National Association of Professional Base Ball Players (NAPBBP), which was in existance from 1871 to 1875 and is considered by some to be the first major league. 

The Western League was founded in 1893, and its aggressive leader Ban Johnson sought to knock the National League from its position of dominance. In 1900, the league changed its name to the American League, and in 1901, it declared the intent to operate as a major league. 

As baseball transitioned into a professional and fully organized sport, many rival leagues sprung up and then disappeared. The most successful rival league was the American Association (1881–1891), which allowed the sale of alcohol at games and is therefore sometimes referred to as the "beer and whiskey league." Two leagues lasted for only a single season but are still considered major leagues by historians due to their high caliber of players and play: the Union Association (1884) and the Players' League (1890). As hinted at by their names, these leagues were attempts to return to player-controlled leagues that did not enforce the hated reserve clause. This clause stated that players could not freely move to other teams without the consent of the team manager.

In yet another attempt to escape the reserve clause, the Federal League was founded in 1913 and operated as a major league in 1914-1915. The Federal League was the last independent league to play on a major league baseball field. 

## Adding data to the database

To add the data currently in csv format to a normalized database, we need a single column that can be used as the primary key. However, the game log file does not contain a single column that can be used to uniquely identify each game. We have three options for handling this issue:
1. Make a compound primary key from the existing columns.
2. Create a new column with integer primary keys (1, 2, etc.)
3. Create a new column with a custom format primary key.

Because we're just starting to create the database, it's best to not create a compound key as this could get complicated when we start to join to other tables. Reading through Retrosheet's website, they have a [data dictionary](https://www.retrosheet.org/eventfile.htm) for their files that includes the following unique identifier for games:

> Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, ATL198304080 should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file.

After we import the data, we can create a new column using this convention to act as the primary key.

Now, we need to import the data into SQLite. We have three options for importing this data:

**1. Using the Python SQLite library**

To use this method, first use csv reader to get the data into Python. Next, use the [`Cursor.execute()` method](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute) to create a table. Then, use the `Cursor.executemany()` to insert multiple rows of data in a single command. To prevent [SQL injection attacks](https://en.wikipedia.org/wiki/SQL_injection#Incorrect_type_handling) and maintain correct data types, we should use the `?` placeholder syntax instead of Python string formatting. 

The advantage of this method is that it offers the highest level of control over what's happening. In addition, for larger datasets, we can write a loop that iterates over the csv data line by line so that it isn't all read into memory at once. 

The disadvantage of this method is that is requires a lot of manual data handling. 

**2. Using pandas**

The pandas library contains a useful [to_sql() method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) that can send the contents of a dataframe to a SQLite connection object. To create the table, either we can use the `execute()` method above or pandas will create it for us. Here's an example:

`my_dataframe.to_sql('table_name', sqlite_connection_object, index=False)`

*Note:* We set `index=False` so that pandas doesn't create an extra column for the pandas index. 

The advantage of this method is that can be accomplished in only one or two lines of code, as shown above. However, the disadvantage is that pandas may alter the data as it reads it in and converts the column types automatically. In addition, the dataset must be small enough to be stored in memory using pandas.

**3. From the SQLite shell**

Finally, we can use the SQLite shell to import the csv data:

`sqlite> .mode csv
sqlite> .import filename.csv table_name`

The advantages of this method are that it's fast and that it works well with large data sources. However, this method also has some disadvantages. First, SQLite determines the column types based on the first row of data, which can lead to incorrect types. Second, this method requires SQLite shell access, which isn't always available. Third, if you to create the table yourself, the header needs to be removed from the first line of the csv file; otherwise, SQLite will make the header line the first row of the table.


In this case, since we've already read the data into DataFrames, we'll use the pandas method. The type conversion issue isn't a concern here since we plan to move the data into new tables as we move through the normalization process.

In [2]:
import sqlite3

# Create a function that takes in a SQL query and
# returns a pandas dataframe of the results.
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)

# Create a function that takes a SQL command and executes it
# and also enforces foreign key restriants
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

In [20]:
# Create a new database and a connection to it.

with sqlite3.connect('mlb.db') as conn:

# Create tables for each of the csv files
    games.to_sql('game_log', conn, index=False)
    parks.to_sql('park_codes', conn, index=False)
    people.to_sql('person_codes', conn, index=False)
    teams.to_sql('team_codes', conn, index=False)

ValueError: Table 'game_log' already exists.

In [ ]:
# Create a new column in the game_log table with unique ID
c1 = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''
run_command(c1)

c2 = '''
UPDATE game_log
SET game_id = h_name || date || number_of_game;
'''
run_command(c2)

In [ ]:
# Check the new column.
q1 = '''
SELECT game_id FROM game_log
LIMIT 10;
'''
run_query(q1)

## Data Normalization

Now that the data is all in a SQL database and we have created a primary key for the game_log table, we can start to normalize the data. Specifically, we're going to look for opportunities to reduce repetition. 

 - **game_log table**
  - The columns `v_league` and `h_league` can be removed as this information is contained in the team_codes table.
  - The columns `v_game_number` and `h_game_number` can be deleted since they can be determined from the table.
  - All of the columns that contain umpire, player, and manager names can be removed as this information is contained in the person_codes table. 
 - **park_codes table**
  - The `start` and `end` dates can be derived from the game_log table.
  - The `league` can be derived by finding the home team in the game_log table and then looking up the team in the team_codes table.
 - **person_codes**
  - All four date columns, `player_debut`, `mgr_debut`, `coach_debut`, and `ump_debut`, can be derived from the game_log table. 
 - **team_codes**
  - The `start` and `end` columns can be derived from the game_log table.

In [ ]:
# Check to see if the franch_id column is redundant.
q2 = '''
SELECT * FROM team_codes
WHERE team_id != franch_id
'''
run_query(q2)

**Proposed normalized database schema**

![Normalized database schema](full_schema.png)

**Dataquest normalized database schema**

*Note:* This is the schema that is used for the rest of the project.

![Dataquest normalized database schema](dataquest_schema.svg)

In the normalized `person` table, all of the `debut` columns have been omitted. The `player_debut`, `mgr_debut`, and `ump_debut` can be found in other tables. Since the game log file doesn't contain any data on coaches, the `coach_debut` column was also removed.

In [ ]:
# Create the person table
c3 = '''
CREATE TABLE IF NOT EXISTS person (
person_id TEXT PRIMARY KEY,
first_name TEXT,
last_name TEXT
);
'''
run_command(c3)
# Insert data from the person_codes table
c4 = '''
INSERT OR IGNORE INTO person
SELECT 
    id,
    first,
    last
FROM person_codes;
'''
run_command(c4)
# Confirm that the person table is correct
q3 = '''
SELECT * FROM person
LIMIT 10;
'''
run_query(q3)

The `start`, `end`, and `league` columns were removed from the normalized `park` table.
 - The `start` and `end` dates can be derived from the game_log table.
 - The `league` can be derived by finding the home team in the game_log table and then looking up the team in the team_codes table.

In [ ]:
# Create the park table
c5 = '''
CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
'''
run_command(c5)

# Insert data from the park_codes table
c6 = '''
INSERT OR IGNORE INTO park
SELECT
    park_id,
    name,
    aka,
    city, 
    state,
    notes
FROM park_codes;
'''
run_command(c6)

# Confirm that the park table is correct
q4 = '''
SELECT * FROM park
LIMIT 10;
'''
run_query(q4)

Because some of the older leagues are not well know, we create a table to store league names.

In [ ]:
# Create the league table
c6 = '''
CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT
);
'''
run_command(c6)

# Insert data manually
c7 = '''
INSERT OR IGNORE INTO league
VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("UA", "Union Association"),
    ("PL", "Players' League"),
    ("FL", "Federal League");
'''
run_command(c7)

# Confirm that the league table is correct
q5 = '''
SELECT * FROM league
'''
run_query(q5)

The new `person_appearance` table includes data on players with positions, umpires, managers, and awards (such as winning pitcher). The `appearance_type` table stores information on the different types of appearances that are available.

This import needs to be done in two steps because the `pd.to_sql()` method does not allow for specifying a primary key. Furthermore, unlike other database systems, e.g., MySQL and PostgreSQL, SQLite does not allow the use of the ALTER TABLE statement to add a primary key to an existing table. Several [workarounds](http://www.sqlitetutorial.net/sqlite-primary-key/) are [suggested](https://stackoverflow.com/questions/39407254/how-to-set-the-primary-key-when-writing-a-pandas-dataframe-to-a-sqlite-database?rq=1) online. 

In [ ]:
# Import data from appearance_type.csv
app_type = pd.read_csv('appearance_type.csv')
with sqlite3.connect('mlb.db') as conn:
    app_type.to_sql('appearance_type', conn, index=False)

In [ ]:
# Confirm that the data imported correctly
q6 = '''
SELECT * FROM appearance_type
'''
run_query(q6)

In [21]:
# Create a new appearance_type table with primary key

# Rename the current table
cw1 = '''
ALTER TABLE appearance_type 
RENAME TO old_table;
'''
run_command(cw1)

# Create the new table with primary key
cw2 = '''
CREATE TABLE appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
'''
run_command(cw2)

# Copy the data from the old table to the new table
cw3 = '''
INSERT INTO appearance_type 
SELECT * FROM old_table;
'''
run_command(cw3)

# Delete the old table
cw4 = '''
DROP TABLE old_table;
'''
run_command(cw4)

# Confirm that the new table is correct
qw1 = '''
SELECT * FROM appearance_type
LIMIT 5;
'''
run_query(qw1)

OperationalError: there is already another table or index with this name: old_table

In [22]:
# Confirm the keys
qw2 = '''
PRAGMA table_info(appearance_type)
'''
run_query(qw2)

,cid,name,type,notnull,dflt_value,pk
0,0,appearance_type_id,TEXT,0,None,1
1,1,name,TEXT,0,None,0
2,2,category,TEXT,0,None,0


For the normalized `team` table, the `start`, `end`, and `seq` columns have been removed.

In [23]:
# Create the team table
c8 = '''
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''
run_command(c8)

# Insert data from the team_codes table
c9 = '''
INSERT OR IGNORE INTO team
SELECT 
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes;
'''
run_command(c9)

# Confirm that the team table is correct
q7 = '''
SELECT * FROM team
LIMIT 10;
'''
run_query(q7)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1
5,BL2,AA,Baltimore,Orioles,BL2
6,BLN,NL,Baltimore,Orioles,BL2
7,BL4,None,Baltimore,Marylands,BL4
8,BLA,AL,Baltimore,Orioles,BLA
9,NYA,AL,New York,Yankees,BLA


The normalized `game` table includes all of the columns that were part of the `game_log` table that don't refer to a specific team or player. In addition, the column with the day of the week was removed since that can derived from the date. The `day_night` column was changed to `day` to make this a boolean column. Even though SQLite doesn't have a separate boolean datatype, we can specify BOOLEAN and have SQLite manage the underlying types behind the scenes.

In [24]:
# Create the game table
c10 = '''
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date INTEGER,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes REAL,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
'''
run_command(c10)

# Insert data from the game_log table
c11 = '''
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    day_night,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
'''
run_command(c11)

# Confirm that the game table is correct
q8 = '''
SELECT * FROM game
LIMIT 10
'''
run_query(q8)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,D,None,None,None,200.0,120.0,None,Y
1,WS3187105050,18710505,0,WAS01,54,D,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,D,None,None,None,1000.0,140.0,None,Y
3,CH1187105080,18710508,0,CHI01,54,D,None,None,None,5000.0,150.0,None,Y
4,TRO187105090,18710509,0,TRO01,54,D,None,None,None,3250.0,145.0,HTBF,Y
5,CL1187105110,18710511,0,CLE01,48,D,None,V,None,2500.0,120.0,None,Y
6,CL1187105130,18710513,0,CIN01,54,D,None,None,None,1200.0,150.0,None,Y
7,FW1187105130,18710513,0,FOR01,54,D,None,None,None,1500.0,105.0,None,Y
8,FW1187105150,18710515,0,FOR01,54,D,None,None,None,NaN,140.0,None,Y
9,BS1187105160,18710516,0,BOS01,54,D,None,None,None,2500.0,NaN,HTBF,Y


The `team_appearance` table contains all of the team data for each game. This table uses a compound primary key of `team_id` and `game_id`. A new boolean column `home` is added to differentiate between the home and visiting teams. The remainder of the columns are team scores or statistics that are repeated in the original `game_log` table for the home and visiting teams.

In [25]:
# Use a query to extract the schema from the game_log table
q9 = '''
SELECT sql FROM sqlite_master
WHERE name = "game_log"
AND type = "table";
'''
run_query(q9)

sql
0  CREATE TABLE "game_log" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_playe

In [26]:
# Create the team_appearance table
c12 = '''
CREATE TABLE IF NOT EXISTS team_appearance(
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats REAL,
    hits REAL, 
    doubles REAL, 
    triples REAL, 
    homeruns REAL, 
    rbi REAL, 
    sacrifice_hits REAL, 
    sacrifice_flies REAL, 
    hit_by_pitch REAL, 
    walks REAL, 
    intentional_walks REAL, 
    strikeouts REAL, 
    stolen_bases REAL, 
    caught_stealing REAL, 
    grounded_into_double REAL, 
    first_catcher_interference REAL, 
    left_on_base REAL, 
    pitchers_used REAL, 
    individual_earned_runs REAL, 
    team_earned_runs REAL, 
    wild_pitches REAL, 
    balks REAL, 
    putouts REAL, 
    assists REAL,
    errors REAL, 
    passed_balls REAL, 
    double_plays REAL,
    triple_plays REAL,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''
run_command(c12)

c13 = '''
INSERT OR IGNORE INTO team_appearance
SELECT
    v_name,
    game_id,
    0 AS home,
    v_league,
    v_score,
    v_line_score, 
    v_at_bats, 
    v_hits, 
    v_doubles, 
    v_triples, 
    v_homeruns, 
    v_rbi, 
    v_sacrifice_hits, 
    v_sacrifice_flies, 
    v_hit_by_pitch, 
    v_walks, 
    v_intentional_walks, 
    v_strikeouts, 
    v_stolen_bases, 
    v_caught_stealing, 
    v_grounded_into_double, 
    v_first_catcher_interference, 
    v_left_on_base, 
    v_pitchers_used, 
    v_individual_earned_runs, 
    v_team_earned_runs, 
    v_wild_pitches, 
    v_balks, 
    v_putouts, 
    v_assists,
    v_errors, 
    v_passed_balls, 
    v_double_plays,
    v_triple_plays
FROM game_log
UNION
SELECT
    h_name,
    game_id,
    1 AS home,
    h_league,
    h_score,
    h_line_score, 
    h_at_bats, 
    h_hits, 
    h_doubles, 
    h_triples, 
    h_homeruns, 
    h_rbi, 
    h_sacrifice_hits, 
    h_sacrifice_flies, 
    h_hit_by_pitch, 
    h_walks, 
    h_intentional_walks, 
    h_strikeouts, 
    h_stolen_bases, 
    h_caught_stealing, 
    h_grounded_into_double, 
    h_first_catcher_interference, 
    h_left_on_base, 
    h_pitchers_used, 
    h_individual_earned_runs, 
    h_team_earned_runs, 
    h_wild_pitches, 
    h_balks, 
    h_putouts, 
    h_assists,
    h_errors, 
    h_passed_balls, 
    h_double_plays,
    h_triple_plays
FROM game_log;
'''
run_command(c13)

# Confirm that the team_appearance table is correct
# Select more recent games to show data other than "none"
q10 = '''
SELECT * FROM team_appearance
WHERE game_id LIKE "%2016%"
LIMIT 10
'''
run_query(q10)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ANA,ANA201604040,1,AL,0,000000000,29.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,0.0,0.0,0.0,0.0,3.0,5.0,9.0,9.0,0.0,0.0,27.0,8.0,1.0,0.0,1.0,0.0
1,ANA,ANA201604050,1,AL,1,000001000,32.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,4.0,5.0,6.0,6.0,0.0,0.0,27.0,9.0,0.0,0.0,0.0,0.0
2,ANA,ANA201604070,1,AL,4,001020001,36.0,11.0,2.0,0.0,1.0,4.0,0.0,0.0,0.0,4.0,1.0,7.0,0.0,0.0,1.0,0.0,10.0,5.0,2.0,2.0,0.0,1.0,27.0,10.0,1.0,0.0,0.0,0.0
3,ANA,ANA201604080,1,AL,3,011100000,33.0,7.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,1.0,0.0,8.0,5.0,7.0,7.0,2.0,0.0,27.0,4.0,1.0,0.0,0.0,0.0
4,ANA,ANA201604090,1,AL,1,100000000,29.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,7.0,0.0,0.0,3.0,0.0,4.0,4.0,2.0,2.0,1.0,0.0,27.0,11.0,2.0,0.0,1.0,0.0
5,ANA,ANA201604100,1,AL,3,00200010x,26.0,7.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,5.0,0.0,1.0,0.0,0.0,3.0,0.0,7.0,4.0,1.0,1.0,0.0,0.0,27.0,6.0,0.0,0.0,1.0,0.0
6,ANA,ANA201604220,1,AL,2,0001100000,34.0,6.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,3.0,6.0,5.0,5.0,0.0,0.0,30.0,12.0,0.0,0.0,1.0,0.0
7,ANA,ANA201604230,1,AL,4,00100201x,30.0,7.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,3.0,1.0,6.0,0.0,1.0,0.0,0.0,5.0,5.0,2.0,2.0,0.0,0.0,27.0,13.0,0.0,1.0,0.0,0.0
8,ANA,ANA201604240,1,AL,4,300000010,31.0,6.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,2.0,0.0,2.0,4.0,8.0,8.0,0.0,0.0,27.0,7.0,3.0,0.0,2.0,0.0
9,ANA,ANA201604250,1,AL,6,30101010x,28.0,8.0,0.0,0.0,3.0,5.0,0.0,0.0,2.0,5.0,0.0,3.0,0.0,1.0,2.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,27.0,7.0,1.0,0.0,0.0,0.0


The final table that we need to create is the `person_appearance` table, which stores information on appearances in games by managers, players, and umpires, as represented in the `appearance_type` table. 

Because creating a compound primary key of every column would be excessively cumbersome, this table will use an integer primary key. By simply adding an integer primary key when creating a new table, [SQLite will autoincrement](https://sqlite.org/autoinc.html) this column if you don't specify a value for the column.

In [7]:
# Create the person_appearance table
c14 = '''
CREATE TABLE IF NOT EXISTS person_appearance(
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES 
        appearance_type(appearance_type_id)
);
'''
run_command(c14)

In [8]:
# Insert data into the person_appearance table

# Start with the offensive and defensive player positions
template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

for hv in ["h", "v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [9]:
# Check the addition of player data.
q10 = '''
SELECT * FROM person_appearance
LIMIT 10;
'''
run_query(q10)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,erstd001,ANA,ANA200004030,D7
1,2,erstd001,ANA,ANA200004030,O1
2,3,erstd001,ANA,ANA200004040,D7
3,4,erstd001,ANA,ANA200004040,O1
4,5,erstd001,ANA,ANA200004050,D7
5,6,erstd001,ANA,ANA200004050,O1
6,7,erstd001,ANA,ANA200004070,D7
7,8,erstd001,ANA,ANA200004070,O1
8,9,erstd001,ANA,ANA200004080,D7
9,10,erstd001,ANA,ANA200004080,O1


In [10]:
# Add the umpires

template_ump = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
    )
    SELECT
        game_id,
        NULL,
        {ump_col},
        "{ump}"
    FROM game_log
    WHERE {ump_col} IS NOT NULL;
'''

ump_abb = ["UHP", "U1B", "U2B", "U3B", "ULF", "URF"]
# Place the column names that begin with numbers inside
# brackets so SQL won't try to interpret these as numbers
ump_cols = ["hp_umpire_id", "[1b_umpire_id]", 
            "[2b_umpire_id]", "[3b_umpire_id]",
            "lf_umpire_id", "rf_umpire_id"]

for num in range(0,6):
    query_vars = {
        "ump": ump_abb[num],
        "ump_col": ump_cols[num]
    }
    run_command(template_ump.format(**query_vars))

In [11]:
# Check the addition of umpire data.
q11 = '''
SELECT * FROM person_appearance
WHERE appearance_type_id = "U2B"
LIMIT 10;
'''
run_query(q11)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,5389091,fergb101,None,BRP189007140,U2B
1,5389092,harrp901,None,CIN192007180,U2B
2,5389093,mccob104,None,BRO192105080,U2B
3,5389094,nalld901,None,WS1192509010,U2B
4,5389095,ormsr901,None,WS1192509020,U2B
5,5389096,ormsr901,None,WS1192509040,U2B
6,5389097,evanb901,None,PHA192509071,U2B
7,5389098,evanb901,None,PHA192509072,U2B
8,5389099,nalld901,None,PHA192509080,U2B
9,5389100,evanb901,None,PHA192509102,U2B


In [12]:
# Add the managers
c15 = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
    )
    
    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL
'''
run_command(c15)

# Check the managers
q12 = '''
SELECT * FROM person_appearance
WHERE appearance_type_id = "MM"
LIMIT 10;
'''
run_query(q12)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,5594566,curte801,ALT,ALT188404300,MM
1,5594567,sullt101,SLU,ALT188404300,MM
2,5594568,curte801,ALT,ALT188405020,MM
3,5594569,sullt101,SLU,ALT188405020,MM
4,5594570,curte801,ALT,ALT188405030,MM
5,5594571,sullt101,SLU,ALT188405030,MM
6,5594572,curte801,ALT,ALT188405050,MM
7,5594573,sullt101,SLU,ALT188405050,MM
8,5594574,curte801,ALT,ALT188405100,MM
9,5594575,murnt101,BSU,ALT188405100,MM


In [13]:
# Add awards and pitcher
c16 = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
    )

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL
'''
run_command(c16)

# Check that the awards were added
q13 = '''
SELECT * FROM person_appearance
WHERE appearance_type_id = "AWB"
LIMIT 5;
'''
run_query(q13)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,5938420,oneip001,NYA,ANA200004030,AWB
1,5938427,willb002,NYA,ANA200004040,AWB
2,5938428,erstd001,ANA,ANA200004050,AWB
3,5938433,andeg001,ANA,ANA200004070,AWB
4,5938441,vaugm001,ANA,ANA200004080,AWB


## Clean up

Now that we have created all of our normalized tables and inserted all of the data, our last task is to remove the original tables that were imported from csv.

In [14]:
c17 = '''
DROP TABLE game_log;
'''
run_command(c17)

c18 = '''
DROP TABLE park_codes;
'''
run_command(c18)

c19 = '''
DROP TABLE team_codes;
'''
run_command(c19)

c20 = '''
DROP TABLE person_codes;
'''
run_command(c20)

## Suggestions for future work

 - Transform the dates into a [SQLite compatible format](https://www.sqlite.org/lang_datefunc.html). 
 - Extract the line scores into innings level data in a new table.
 - Create views to make querying stats easier:
  - season-level stats
  - all-time records
 - Supplement with database with new data:
  - add data from Retrosheet game logs for years after 2016
  - source and add missing pitcher information
  - add player level per-game stats
  - source and include base coach data